In [1]:
import math
import numpy as np
import time

In [2]:
class NN:
    def __init__(self, alpha, etalons, w1, w2, vectors, error, N):
        self.alpha = alpha
        self.etalons = etalons
        self.w1 = w1
        self.w2 = w2
        self.vectors = vectors
        self.error = error
        self.N = N
        self.vector_after_first_layer = 0
        self.vector_after_second_layer = 0

In [3]:
def F(S):
#     result = np.tanh(S)
#     result = 1 / (1 + np.exp(-1*S))
    result = S
    return result

def normalize_m(matrix):
    for j in range(matrix.shape[1]):
        s = 0
        for i in range(matrix.shape[0]):
            s += pow(matrix.item((i,j)), 2)
        s = math.sqrt(s)
        
        for i in range(matrix.shape[0]):
            matrix.itemset((i, j), matrix.item((i, j)) / s)
    return matrix

In [4]:
def derivative(vector_p):
#     return np.subtract(np.array([1] * vector_p.shape[1]), vector_p * vector_p)
    return 1

def transform(numset, p, m):
    vectors = [[0]*p for j in range(m)]
    etalons = [[0] for j in range(m)]
    for i in range(m):
        vectors[i] = numset[i: i + p]   
        etalons[i] = numset[p + i]
    return vectors, etalons


def transform_pred(numset, p, m):
    vectors = [[0]*p for j in range(m)]
    for i in range(m):
        vectors[i] = numset[i: i + p]   
    return vectors


def show_result(model, vectors, p, m):
    vector_after_first_layer = np.zeros(model.w1.shape[1])
    vector_after_second_layer = np.zeros(model.w2.shape[1])

    for i, vector in enumerate(vectors):            
        vector = np.expand_dims(vector, axis=0)
        vector = np.concatenate((vector, vector_after_first_layer, vector_after_second_layer), axis=None)
        first_layer = np.matmul(vector, model.w1)
        vector_after_first_layer = first_layer
        second_layer = np.matmul(vector_after_first_layer, model.w2)
    return second_layer[0]

In [19]:
def run_epoch(model, step = 0):
    time1 = time.time()
    while True:
        errors = []
        vector_after_first_layer = np.zeros(model.w1.shape[1])
        vector_after_second_layer = np.zeros(model.w2.shape[1])
    
        for i, vector in enumerate(model.vectors):            
            vector = np.expand_dims(vector, axis=0)
#             vector_for_w = vector
            vector = np.concatenate((vector, vector_after_first_layer, vector_after_second_layer), axis=None)
            first_layer = np.matmul(vector, model.w1)
            vector_after_first_layer = F(first_layer)
            second_layer = np.matmul(vector_after_first_layer, model.w2)
            vector_after_second_layer = F(second_layer)
            
            d_vector = np.subtract(vector_after_second_layer, model.etalons[i])
            model.w1 = model.w1 - model.alpha * (np.transpose(np.expand_dims(vector, axis=0)) * (d_vector * np.transpose(model.w2)))
            model.w2 = model.w2 - np.transpose(np.expand_dims(model.alpha * (np.transpose(vector_after_first_layer) * d_vector), axis = 0))
            
            errors.append(np.sum(d_vector * d_vector / 2))
        model.w1 = normalize_m(model.w1)      
        model.w2 = normalize_m(model.w2)
        
        step += 1
        sum_error = sum(errors)

        if step % 5000 is 0:
            time2 = time.time()
            print("***step time is ", time2 - time1)
            print(step, " ", sum_error)
            time1 = time2

        if sum_error <= model.error and step > model.N:
#             model.vector_after_first_layer = vector_after_first_layer
#             model.vector_after_second_layer = vector_after_second_layer
            return model

In [20]:
# numset = [ 0.5, 1, 1.5, 0.5, 1, 1.5, 0.5, 1 ]
# numset = [ 0.01, 0.04, 0.09, 0.16, 0.25, 0.36, 0.49, 0.64 ]
# numset = [ 0.001, 0.002, 0.004, 0.008, 0.016, 0.032, 0.064, 0.128 ]
# numset = [ 0.1, 0, 0.1, 0, 0.1, 0, 0.1, 0 ]
# numset = [ 1, 0, 1, 0, 1, 0, 1, 0 ]
# numset = [ 1, 2, 4, 8, 16, 32, 64, 128 ]
# numset = [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8 ]
numset = [ 0.01, 0.01, 0.02, 0.03, 0.05, 0.08, 0.13, 0.21]
# numset = [ 1, 0, -1, 0, 1, 0, -1, 0 ]
# numset = [ 0.001, 0.002, 0.006, 0.024, 0.12, 0.72, 5.04, 40.32 ]
# numset = [1/100, 2/100, 4/100, 8/100, 16/100, 32/100, 64/100]

In [21]:
error = 1000
alpha = 0.1
p = 5
m = 3
N = 100000

vectors, etalons = transform(numset, p, m)
w1 = np.random.rand(p + m + 1, m) * 2 - 1
w2 = np.random.rand(m, 1) * 2 - 1
# print(etalons)
# print(vectors)
model = NN(alpha, etalons, w1, w2, vectors, error, N)
model = run_epoch(model)

***step time is  1.4405369758605957
5000   3.917336196760728e-06
***step time is  1.3854951858520508
10000   9.636851741279102e-07
***step time is  1.339289665222168
15000   2.5689012372839094e-07
***step time is  1.3110311031341553
20000   7.356573910275496e-08
***step time is  1.47825026512146
25000   2.2785713758717802e-08
***step time is  1.5802819728851318
30000   7.69064272314711e-09
***step time is  1.2952017784118652
35000   2.8243897352987096e-09
***step time is  1.3107187747955322
40000   1.114562520393294e-09
***step time is  1.3305447101593018
45000   4.6399013205022136e-10
***step time is  1.6611313819885254
50000   2.0018409799562559e-10
***step time is  1.7205357551574707
55000   8.829757265116153e-11
***step time is  1.3136157989501953
60000   3.9454958181460694e-11
***step time is  1.3590707778930664
65000   1.7759957805054962e-11
***step time is  1.436324119567871
70000   8.026653801271762e-12
***step time is  1.3211073875427246
75000   3.6354981960610974e-12
***step 

In [22]:
numset_for_predict = [  0.01, 0.02, 0.03, 0.05, 0.08, 0.13, 0.21]
vectors = transform_pred(numset_for_predict, p, m)
result = show_result(model, vectors, p, m)
print(vectors[m-1])
print(result)

[0.03, 0.05, 0.08, 0.13, 0.21]
0.3390182793129073
